In [18]:
import pandas as pd

# Memuat dataset
dataset = pd.read_csv('C:\\Users\\ASUS\\Documents\\Crop_recommendation.csv')

print(dataset.head())



    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


In [19]:
dataset.drop(columns=['temperature','rainfall'], inplace=True)


In [20]:
dataset = dataset[['N', 'P', 'K', 'humidity', 'ph', 'label']]

# Mengganti 'Humidity' dengan 'moisture'
dataset.rename(columns={'Humidity': 'moisture'}, inplace=True)

In [30]:
# Memisahkan kolom numerik dan kategorikal
numeric_columns = dataset.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = dataset.select_dtypes(include=['object']).columns

# Mengisi nilai yang hilang pada kolom numerik dengan median
dataset[numeric_columns] = dataset[numeric_columns].fillna(dataset[numeric_columns].median())

# Mengisi nilai yang hilang pada kolom kategorikal dengan mode
for col in categorical_columns:
    dataset[col].fillna(dataset[col].mode()[0], inplace=True)


In [31]:
# Mengubah kolom kategorikal menjadi dummy/indicator variables
dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)


In [ ]:
### Menambah Kolom Target

In [32]:
def calculate_nitrogen_need(row):
    return 2.5 * row['N']  # Formula sintetik

def calculate_phosphorus_need(row):
    return 2.5 * row['P']  # Formula sintetik

def calculate_potassium_need(row):
    return 2.5 * row['K']  # Formula sintetik

def calculate_water_need(row):
    return 4.6 * row['moisture'] 

dataset['nitrogen_need'] = dataset.apply(calculate_nitrogen_need, axis=1)
dataset['phosphorus_need'] = dataset.apply(calculate_phosphorus_need, axis=1)
dataset['potassium_need'] = dataset.apply(calculate_potassium_need, axis=1)
dataset['water_need'] = dataset.apply(calculate_water_need, axis=1)

In [33]:
# Fitur
X = dataset[['N', 'P', 'K', 'moisture', 'ph']]

# Target untuk setiap jenis pupuk dan air
y_nitrogen = dataset['nitrogen_need']
y_phosphorus = dataset['phosphorus_need']
y_potassium = dataset['potassium_need']
y_water = dataset['water_need']


In [ ]:
### Pembagian Data

In [34]:
from sklearn.model_selection import train_test_split

# Membagi data menjadi data latih dan data uji untuk nitrogen
X_train_nitrogen, X_test_nitrogen, y_train_nitrogen, y_test_nitrogen = train_test_split(X, y_nitrogen, test_size=0.2, random_state=42)

# Membagi data menjadi data latih dan data uji untuk phosphorus
X_train_phosphorus, X_test_phosphorus, y_train_phosphorus, y_test_phosphorus = train_test_split(X, y_phosphorus, test_size=0.2, random_state=42)

# Membagi data menjadi data latih dan data uji untuk potassium
X_train_potassium, X_test_potassium, y_train_potassium, y_test_potassium = train_test_split(X, y_potassium, test_size=0.2, random_state=42)

# Membagi data menjadi data latih dan data uji untuk air
X_train_water, X_test_water, y_train_water, y_test_water = train_test_split(X, y_water, test_size=0.2, random_state=42)


In [ ]:
### Pemodelan

In [35]:
from sklearn.tree import DecisionTreeRegressor

# Model untuk nitrogen
model_nitrogen = DecisionTreeRegressor(random_state=42)
model_nitrogen.fit(X_train_nitrogen, y_train_nitrogen)

# Model untuk phosphorus
model_phosphorus = DecisionTreeRegressor(random_state=42)
model_phosphorus.fit(X_train_phosphorus, y_train_phosphorus)

# Model untuk potassium
model_potassium = DecisionTreeRegressor(random_state=42)
model_potassium.fit(X_train_potassium, y_train_potassium)

# Model untuk air
model_water = DecisionTreeRegressor(random_state=42)
model_water.fit(X_train_water, y_train_water)


DecisionTreeRegressor(random_state=42)

In [36]:
# Prediksi untuk nitrogen
y_pred_nitrogen = model_nitrogen.predict(X_test_nitrogen)

# Prediksi untuk phosphorus
y_pred_phosphorus = model_phosphorus.predict(X_test_phosphorus)

# Prediksi untuk potassium
y_pred_potassium = model_potassium.predict(X_test_potassium)

# Prediksi untuk air
y_pred_water = model_water.predict(X_test_water)


In [37]:
### Evaluasi model

In [38]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluasi model nitrogen
mse_nitrogen = mean_squared_error(y_test_nitrogen, y_pred_nitrogen)
r2_nitrogen = r2_score(y_test_nitrogen, y_pred_nitrogen)
print(f"Nitrogen - Mean Squared Error: {mse_nitrogen}")
print(f"Nitrogen - R^2 Score: {r2_nitrogen}")

# Evaluasi model phosphorus
mse_phosphorus = mean_squared_error(y_test_phosphorus, y_pred_phosphorus)
r2_phosphorus = r2_score(y_test_phosphorus, y_pred_phosphorus)
print(f"Phosphorus - Mean Squared Error: {mse_phosphorus}")
print(f"Phosphorus - R^2 Score: {r2_phosphorus}")

# Evaluasi model potassium
mse_potassium = mean_squared_error(y_test_potassium, y_pred_potassium)
r2_potassium = r2_score(y_test_potassium, y_pred_potassium)
print(f"Potassium - Mean Squared Error: {mse_potassium}")
print(f"Potassium - R^2 Score: {r2_potassium}")

# Evaluasi model air
mse_water = mean_squared_error(y_test_water, y_pred_water)
r2_water = r2_score(y_test_water, y_pred_water)
print(f"Water - Mean Squared Error: {mse_water}")
print(f"Water - R^2 Score: {r2_water}")


Nitrogen - Mean Squared Error: 0.07086167800453515
Nitrogen - R^2 Score: 0.9999913448673293
Phosphorus - Mean Squared Error: 0.04251700680272109
Phosphorus - R^2 Score: 0.9999937434071685
Potassium - Mean Squared Error: 0.0
Potassium - R^2 Score: 1.0
Water - Mean Squared Error: 0.0
Water - R^2 Score: 1.0


In [41]:
from joblib import dump

dump(scaler, 'scaler.joblib')

# Simpan model nitrogen
dump(model_nitrogen, 'model_nitrogen.joblib')

# Simpan model phosphorus
dump(model_phosphorus, 'model_phosphorus.joblib')

# Simpan model potassium
dump(model_potassium, 'model_potassium.joblib')

# Simpan model water
dump(model_water, 'model_water.joblib')


['model_water.joblib']

In [44]:
print("Fitur yang digunakan dalam scaler:", scaler.feature_names_in_)


Fitur yang digunakan dalam scaler: ['N' 'P' 'K' 'moisture' 'ph']


In [45]:
# Ubah nama kolom dari 'humidity' menjadi 'moisture'
sensor_df = sensor_df.rename(columns={'humidity': 'moisture'})


In [46]:
# Pastikan urutan fitur sesuai dengan yang digunakan saat fit scaler
expected_features = ['N', 'P', 'K', 'moisture', 'ph']
sensor_df = sensor_df[expected_features]


In [47]:
# Normalisasi data sensor yang telah disesuaikan
sensor_normalized = scaler.transform(sensor_df)


In [48]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib  # Ubah dari pickle ke joblib jika Anda menggunakan joblib

# Inisialisasi aplikasi Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate("kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Mengambil data dari Firebase
ref = db.reference('sensorData')
sensor_data = ref.get()

# Cek apakah data adalah nested dictionary dan ambil nilai dari 'sensorData' jika ada
if isinstance(sensor_data, dict) and 'sensorData' in sensor_data:
    sensor_data = sensor_data['sensorData']

# Konversi data menjadi DataFrame
sensor_df = pd.DataFrame([sensor_data])

# Ubah nama kolom dari 'humidity' menjadi 'moisture'
sensor_df = sensor_df.rename(columns={'humidity': 'moisture'})

# Ubah tipe data dari kolom menjadi float jika memungkinkan
try:
    sensor_df = sensor_df.astype({'K': 'float', 'N': 'float', 'P': 'float', 'moisture': 'float', 'ph': 'float'})
except KeyError as e:
    print(f"KeyError: {e}. Available columns: {sensor_df.columns}")

# Pastikan urutan fitur sesuai dengan model
expected_features = ['N', 'P', 'K', 'moisture', 'ph']
sensor_df = sensor_df[expected_features]

# Muat model dan scaler dari file menggunakan joblib
scaler = joblib.load('scaler.joblib')
model_water = joblib.load('model_water.joblib')
model_n = joblib.load('model_nitrogen.joblib')
model_p = joblib.load('model_phosphorus.joblib')
model_k = joblib.load('model_potassium.joblib')

# Pastikan fitur dalam scaler sesuai dengan yang digunakan pada model
print("Fitur yang digunakan dalam scaler:", scaler.feature_names_in_)

# Normalisasi data sensor yang telah disesuaikan
sensor_normalized = scaler.transform(sensor_df)

# Prediksi kebutuhan air dan pupuk
predicted_water = model_water.predict(sensor_normalized)
predicted_n = model_n.predict(sensor_normalized)
predicted_p = model_p.predict(sensor_normalized)
predicted_k = model_k.predict(sensor_normalized)

# Update prediksi ke Firebase
predictions_ref = db.reference('predictions_realtime')
predictions_ref.set({
    'Penambahan Air (ml)': predicted_water[0],
    'Penambahan Pupuk KCL (K)': predicted_k[0],
    'Penambahan Pupuk SP36 (P)': predicted_p[0],
    'Penambahan Pupuk Urea (N)': predicted_n[0],
})

print(f'Predicted Water Requirement: {predicted_water[0]}')
print(f'Predicted N Requirement: {predicted_n[0]}')
print(f'Predicted P Requirement: {predicted_p[0]}')
print(f'Predicted K Requirement: {predicted_k[0]}')


Fitur yang digunakan dalam scaler: ['N' 'P' 'K' 'moisture' 'ph']


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


Predicted Water Requirement: 276.0
Predicted N Requirement: 0.0
Predicted P Requirement: 12.5
Predicted K Requirement: 12.5


In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Misalkan y_true adalah nilai aktual dan y_pred adalah hasil prediksi
y_true = np.array([82.5, 25, 97.5, 322.0])  # Contoh nilai aktual
y_pred = np.array([0.0, 12.5, 12.5, 276.0])  # Contoh hasil prediksi

# Hitung metrik error
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")


Mean Absolute Error (MAE): 56.5
Mean Squared Error (MSE): 4075.875
Root Mean Squared Error (RMSE): 63.84257983509125
R-squared (R²): 0.6815053941989773


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import dump

# **1. Load Dataset**
dataset = pd.read_csv('C:\\Users\\ASUS\\Documents\\Crop_recommendation.csv')

# **2. Preprocessing**
# Drop unnecessary columns
dataset.drop(columns=['temperature', 'rainfall'], inplace=True)

# Rename 'Humidity' to 'moisture'
dataset.rename(columns={'humidity': 'moisture'}, inplace=True)

# Fill missing values
numeric_columns = dataset.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = dataset.select_dtypes(include=['object']).columns

dataset[numeric_columns] = dataset[numeric_columns].fillna(dataset[numeric_columns].median())

for col in categorical_columns:
    dataset[col].fillna(dataset[col].mode()[0], inplace=True)

# Convert categorical columns to dummy variables
dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)

# **3. Feature Engineering**
def calculate_nitrogen_need(row):
    return 2.5 * row['N']

def calculate_phosphorus_need(row):
    return 2.5 * row['P']

def calculate_potassium_need(row):
    return 2.5 * row['K']

def calculate_water_need(row):
    return 4.6 * row['moisture']

dataset['nitrogen_need'] = dataset.apply(calculate_nitrogen_need, axis=1)
dataset['phosphorus_need'] = dataset.apply(calculate_phosphorus_need, axis=1)
dataset['potassium_need'] = dataset.apply(calculate_potassium_need, axis=1)
dataset['water_need'] = dataset.apply(calculate_water_need, axis=1)

# **4. Split Data**
X = dataset[['N', 'P', 'K', 'moisture', 'ph']]
y_nitrogen = dataset['nitrogen_need']
y_phosphorus = dataset['phosphorus_need']
y_potassium = dataset['potassium_need']
y_water = dataset['water_need']

X_train_nitrogen, X_test_nitrogen, y_train_nitrogen, y_test_nitrogen = train_test_split(X, y_nitrogen, test_size=0.2, random_state=42)
X_train_phosphorus, X_test_phosphorus, y_train_phosphorus, y_test_phosphorus = train_test_split(X, y_phosphorus, test_size=0.2, random_state=42)
X_train_potassium, X_test_potassium, y_train_potassium, y_test_potassium = train_test_split(X, y_potassium, test_size=0.2, random_state=42)
X_train_water, X_test_water, y_train_water, y_test_water = train_test_split(X, y_water, test_size=0.2, random_state=42)

# **5. Train Models**
model_nitrogen = DecisionTreeRegressor(random_state=42)
model_phosphorus = DecisionTreeRegressor(random_state=42)
model_potassium = DecisionTreeRegressor(random_state=42)
model_water = DecisionTreeRegressor(random_state=42)

model_nitrogen.fit(X_train_nitrogen, y_train_nitrogen)
model_phosphorus.fit(X_train_phosphorus, y_train_phosphorus)
model_potassium.fit(X_train_potassium, y_train_potassium)
model_water.fit(X_train_water, y_train_water)

# **6. Hyperparameter Tuning**
# Define parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Apply GridSearchCV for each model
grid_search_nitrogen = GridSearchCV(estimator=model_nitrogen, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search_phosphorus = GridSearchCV(estimator=model_phosphorus, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search_potassium = GridSearchCV(estimator=model_potassium, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')
grid_search_water = GridSearchCV(estimator=model_water, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# Fit grid search
grid_search_nitrogen.fit(X_train_nitrogen, y_train_nitrogen)
grid_search_phosphorus.fit(X_train_phosphorus, y_train_phosphorus)
grid_search_potassium.fit(X_train_potassium, y_train_potassium)
grid_search_water.fit(X_train_water, y_train_water)

# Best parameters
print("Best parameters for nitrogen:", grid_search_nitrogen.best_params_)
print("Best parameters for phosphorus:", grid_search_phosphorus.best_params_)
print("Best parameters for potassium:", grid_search_potassium.best_params_)
print("Best parameters for water:", grid_search_water.best_params_)

# **7. Evaluate Models**
# Use best models
best_model_nitrogen = grid_search_nitrogen.best_estimator_
best_model_phosphorus = grid_search_phosphorus.best_estimator_
best_model_potassium = grid_search_potassium.best_estimator_
best_model_water = grid_search_water.best_estimator_

# Predictions
y_pred_nitrogen = best_model_nitrogen.predict(X_test_nitrogen)
y_pred_phosphorus = best_model_phosphorus.predict(X_test_phosphorus)
y_pred_potassium = best_model_potassium.predict(X_test_potassium)
y_pred_water = best_model_water.predict(X_test_water)

# Calculate and print performance metrics
# Nitrogen
mae_nitrogen = mean_absolute_error(y_test_nitrogen, y_pred_nitrogen)
mse_nitrogen = mean_squared_error(y_test_nitrogen, y_pred_nitrogen)
r2_nitrogen = r2_score(y_test_nitrogen, y_pred_nitrogen)

# Phosphorus
mae_phosphorus = mean_absolute_error(y_test_phosphorus, y_pred_phosphorus)
mse_phosphorus = mean_squared_error(y_test_phosphorus, y_pred_phosphorus)
r2_phosphorus = r2_score(y_test_phosphorus, y_pred_phosphorus)

# Potassium
mae_potassium = mean_absolute_error(y_test_potassium, y_pred_potassium)
mse_potassium = mean_squared_error(y_test_potassium, y_pred_potassium)
r2_potassium = r2_score(y_test_potassium, y_pred_potassium)

# Water
mae_water = mean_absolute_error(y_test_water, y_pred_water)
mse_water = mean_squared_error(y_test_water, y_pred_water)
r2_water = r2_score(y_test_water, y_pred_water)

# Overall Performance
mae_overall = (mae_nitrogen + mae_phosphorus + mae_potassium + mae_water) / 4
mse_overall = (mse_nitrogen + mse_phosphorus + mse_potassium + mse_water) / 4
r2_overall = (r2_nitrogen + r2_phosphorus + r2_potassium + r2_water) / 4

print(f"Overall - Mean Absolute Error: {mae_overall}")
print(f"Overall - Mean Squared Error: {mse_overall}")
print(f"Overall - R^2 Score: {r2_overall}")

# **8. Save Models**
dump(best_model_nitrogen, 'model_nitrogen.joblib')
dump(best_model_phosphorus, 'model_phosphorus.joblib')
dump(best_model_potassium, 'model_potassium.joblib')
dump(best_model_water, 'model_water.joblib')


Best parameters for nitrogen: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best parameters for phosphorus: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best parameters for potassium: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best parameters for water: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Overall - Mean Absolute Error: 0.07240600071189629
Overall - Mean Squared Error: 0.084536228057739
Overall - R^2 Score: 0.999991216617317


['model_water.joblib']

In [7]:
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import joblib  # Ubah dari pickle ke joblib jika Anda menggunakan joblib

# Inisialisasi aplikasi Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate("kebuncabe-6f515-firebase-adminsdk-n54zr-c89d1f51dd.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://kebuncabe-6f515-default-rtdb.firebaseio.com/'
    })

# Mengambil data dari Firebase
ref = db.reference('sensorData')
sensor_data = ref.get()

# Cek apakah data adalah nested dictionary dan ambil nilai dari 'sensorData' jika ada
if isinstance(sensor_data, dict) and 'sensorData' in sensor_data:
    sensor_data = sensor_data['sensorData']

# Konversi data menjadi DataFrame
sensor_df = pd.DataFrame([sensor_data])

# Ubah nama kolom dari 'humidity' menjadi 'moisture'
sensor_df = sensor_df.rename(columns={'humidity': 'moisture'})

# Ubah tipe data dari kolom menjadi float jika memungkinkan
try:
    sensor_df = sensor_df.astype({'K': 'float', 'N': 'float', 'P': 'float', 'moisture': 'float', 'ph': 'float'})
except KeyError as e:
    print(f"KeyError: {e}. Available columns: {sensor_df.columns}")

# Pastikan urutan fitur sesuai dengan model
expected_features = ['N', 'P', 'K', 'moisture', 'ph']
sensor_df = sensor_df[expected_features]

# Muat model dan scaler dari file menggunakan joblib
scaler = joblib.load('scaler.joblib')
model_water = joblib.load('model_water.joblib')
model_n = joblib.load('model_nitrogen.joblib')
model_p = joblib.load('model_phosphorus.joblib')
model_k = joblib.load('model_potassium.joblib')

# Pastikan fitur dalam scaler sesuai dengan yang digunakan pada model
print("Fitur yang digunakan dalam scaler:", scaler.feature_names_in_)

# Normalisasi data sensor yang telah disesuaikan
sensor_normalized = scaler.transform(sensor_df)

# Prediksi kebutuhan air dan pupuk
predicted_water = model_water.predict(sensor_normalized)
predicted_n = model_n.predict(sensor_normalized)
predicted_p = model_p.predict(sensor_normalized)
predicted_k = model_k.predict(sensor_normalized)

# Update prediksi ke Firebase
predictions_ref = db.reference('predictions_realtime')
predictions_ref.set({
    'Penambahan Air (ml)': predicted_water[0],
    'Penambahan Pupuk KCL (K)': predicted_k[0],
    'Penambahan Pupuk SP36 (P)': predicted_p[0],
    'Penambahan Pupuk Urea (N)': predicted_n[0],
})

print(f'Predicted Water Requirement: {predicted_water[0]}')
print(f'Predicted N Requirement: {predicted_n[0]}')
print(f'Predicted P Requirement: {predicted_p[0]}')
print(f'Predicted K Requirement: {predicted_k[0]}')


Fitur yang digunakan dalam scaler: ['N' 'P' 'K' 'moisture' 'ph']


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


Predicted Water Requirement: 66.35304615
Predicted N Requirement: 15.0
Predicted P Requirement: 15.0
Predicted K Requirement: 12.5


In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Prediksi dan nilai aktual
y_pred_nitrogen = best_model_nitrogen.predict(X_test_nitrogen)
y_pred_phosphorus = best_model_phosphorus.predict(X_test_phosphorus)
y_pred_potassium = best_model_potassium.predict(X_test_potassium)
y_pred_water = best_model_water.predict(X_test_water)

# Gabungkan semua prediksi dan nilai aktual
y_test_all = np.concatenate([y_test_nitrogen, y_test_phosphorus, y_test_potassium, y_test_water])
y_pred_all = np.concatenate([y_pred_nitrogen, y_pred_phosphorus, y_pred_potassium, y_pred_water])

# Menghitung MAE, MSE, dan R²
mae_all = mean_absolute_error(y_test_all, y_pred_all)
mse_all = mean_squared_error(y_test_all, y_pred_all)
r2_all = r2_score(y_test_all, y_pred_all)

# Cetak hasil
print(f"Overall - Mean Absolute Error: {mae_all}")
print(f"Overall - Mean Squared Error: {mse_all}")
print(f"Overall - R^2 Score: {r2_all}")

# Menghitung akurasi berdasarkan kesalahan absolut < 10% dari nilai sebenarnya
threshold = 0.1  # 10%
accuracy_within_threshold = np.mean(np.abs((y_pred_all - y_test_all) / y_test_all) < threshold)
print(f"Accuracy within 10% threshold: {accuracy_within_threshold * 100:.2f}%")

# Menghitung akurasi berdasarkan kesalahan absolut < nilai tertentu
absolute_error_threshold = 0.1  # Contoh nilai tertentu
accuracy_within_absolute_error = np.mean(np.abs(y_pred_all - y_test_all) < absolute_error_threshold)
print(f"Accuracy within absolute error threshold {absolute_error_threshold}: {accuracy_within_absolute_error * 100:.2f}%")


Overall - Mean Absolute Error: 0.07240600071189629
Overall - Mean Squared Error: 0.08453622805773899
Overall - R^2 Score: 0.9999952359941653
Accuracy within 10% threshold: 99.72%
Accuracy within absolute error threshold 0.1: 85.40%


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2252\2903222809.py:26: RuntimeWarning: invalid value encountered in divide
  accuracy_within_threshold = np.mean(np.abs((y_pred_all - y_test_all) / y_test_all) < threshold)
